# Getting Started with a Movie Recommendation System

Reference: https://www.kaggle.com/code/ibtesama/getting-started-with-a-movie-recommendation-system/notebook?select=tmdb_5000_credits.csv


## Prerequisites

- Install Dependencies
- Install Dataset


In [ ]:
%pip install kagglehub
%pip install pandas
%pip install numpy==1.26
%pip install matplotlib
%pip install scikit-learn
%pip install scikit-surprise

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tmdb/tmdb-movie-metadata")

print("Path to dataset files:", path)

## Approaches

### Demographic Filtering

- 영화 인기도나 장로에 기반하여 모든 사용자에게 일반화된 추천 제공
- 인구통계학적으로 비슷한 사용자에게는 같은 영화 추천
- 더 인기있는 영화가 평균적인 대중들이 좋아할 확률이 더 높을 것이라는 가정에 기반

### Content Based Filtering

- 특정 아이템에 기반하여 비슷한 아이템을 추천
- 장르, 감독, 설명, 배우 등의 메타데이터 사용
- 특정 영화를 좋아하는 사람이러면 비슷한 영화도 좋아할 것이라는 가정에 기반

### Collaborative Filtering

- 비슷한 관심사를 가진 사람들을 매칭하여 추천 제공
- Content Based Filter과 달리 메타데이터를 사용하지 않음


## Load Data

- 앞서 다운로드 받은 데이터를 Pandas DataFrame으로 로드


In [ ]:
import pandas as pd

df1 = pd.read_csv(f"{path}/tmdb_5000_credits.csv")
df2 = pd.read_csv(f"{path}/tmdb_5000_movies.csv")

In [ ]:
# 데이터셋 살펴보기: tmdb_5000_credits.csv
df1

In [ ]:
# 데이터셋 살펴보기: tmdb_5000_movies.csv
df2

## Dataset Join

- `df1`의 일부 컬럼만 뽑아 `df2`와 조인


In [ ]:
df1.columns = ["id", "tittle", "cast", "crew"]
df2 = df2.merge(df1, on="id")

# 5개의 Row 뽑아보기
df2.head(5)

## Demographic Filtering

### TODO

- 영화의 점수를 매길 수 있는 Metric 필요
- 모든 영화의 점수 계산
- 점수에 따라 정렬하고, 사용자에게 가장 높은 점수의 영화 추천

### Metric

- 8.9점이지만 3명만 평가한 영화와 7.8점이지만 40만명이 평가한 영화 중 어느 것이 더 좋은 영화일까?
- 이는 알 수 없으므로, IMDB의 가중 평점(WR) 사용

$$
WR = \left( \frac{v}{v+m} \cdot R \right) + \left( \frac{m}{v+m} \cdot C \right)
$$

- $v$ : 영화에 대한 평가 수
- $m$ : 최소 평가 수
- $R$ : 영화 평균 평점
- $C$ : 전체 영화 평균 평점

이미 $v$와 $R$은 주어져 있으므로, $C$와 $m$을 정의해야 함


In [ ]:
C = df2["vote_average"].mean()
C

- 90th Percentile을 사용하여 $m$ 정의
- 영화가 차트에 추천되기 위해서는, 최소 90%의 영화보다 더 많은 평가를 받아야 함


In [ ]:
m = df2["vote_count"].quantile(0.9)
m

- 대상이 되는 영화 필터링
- 481개의 영화만 추천


In [ ]:
q_movies = df2.copy().loc[df2["vote_count"] >= m]
q_movies.shape

- Weighted Rating 계산


In [ ]:
def weighted_rating(x, m=m, C=C):
    v = x["vote_count"]
    R = x["vote_average"]

    return (v / (v + m) * R) + (m / (m + v) * C)

In [ ]:
q_movies["score"] = q_movies.apply(weighted_rating, axis=1)

- Score를 기반으로 정렬


In [ ]:
q_movies = q_movies.sort_values("score", ascending=False)

q_movies[["title", "vote_count", "vote_average", "score"]].head(10)

In [ ]:
pop = df2.sort_values("popularity", ascending=False)

import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.barh(
    pop["title"].head(6), pop["popularity"].head(6), align="center", color="skyblue"
)
plt.gca().invert_yaxis()
plt.xlabel("Popularity")
plt.title("Popular Movies")

## Content Based Filtering

- 영화의 개요, 감독, 배우, 장르 등을 사용하여 영화 간 유사성 계산
- 유사성 점수를 기반으로 추천
- `overvie` 컬럼을 사용하여 영화 간 유사성 계산
- Term Frequency-Inverse Document Frequency(TF-IDF) 사용


In [ ]:
# Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

# Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words="english")

# Replace NaN with an empty string
df2["overview"] = df2["overview"].fillna("")

# Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df2["overview"])

# Output the shape of tfidf_matrix
tfidf_matrix.shape

- Cosine Similarity 사용하여 유사성 계산

$$
cosine(x, y) = \frac{x \cdot y}{||x|| \cdot ||y||}
$$

- TF-IDF vectorizer를 사용했기 때문에, Dot Product가 Cosine Similarity와 동일


In [ ]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
indices = pd.Series(df2.index, index=df2["title"]).drop_duplicates()

- Recommendation 함수 구현
- 영화 제목으로부터 인덱스 추출
- 모든 영화와의 Cosine Similarity 계산하고, Position과 Score 튜플로 변환
- Score에 따라 정렬하고, 상위 10개 추천


In [ ]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df2["title"].iloc[movie_indices]

In [ ]:
get_recommendations("The Dark Knight Rises")

- Content Based Filtering은 더 좋은 메타데이터를 사용할수록 더 좋은 결과를 제공
- Top 3 배우, 감독, 연관 장르, 영화 개요를 메타데이터로 사용


In [ ]:
# Parse the stringified features into their corresponding python objects
from ast import literal_eval

features = ["cast", "crew", "keywords", "genres"]
for feature in features:
    df2[feature] = df2[feature].apply(literal_eval)

In [ ]:
import numpy as np


# Get the director's name from the crew feature. If director is not listed, return NaN
def get_director(x):
    for i in x:
        if i["job"] == "Director":
            return i["name"]
    return np.nan

In [ ]:
# Returns the list top 3 elements or entire list; whichever is more.
def get_list(x):
    if isinstance(x, list):
        names = [i["name"] for i in x]
        # Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    # Return empty list in case of missing/malformed data
    return []

In [ ]:
# Define new director, cast, genres and keywords features that are in a suitable form.
df2['director'] = df2['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(get_list)

In [ ]:
# Print the new features of the first 3 films
df2[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

- 이름과 Keyword를 소문자로 변환하고 띄어쓰기 제거
- Vectorizer가 "Johnny Depp"와 "Johnny Galecki"의 Johnny를 동일하게 처리하지 않도록 하기 위함

In [ ]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return 

In [ ]:
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    df2[feature] = df2[feature].apply(clean_data)

- 모든 Metatdata를 하나의 문자열로 결합
- 이를 Vectorizer에 적용

In [ ]:
def create_soup(x):
    if not x["director"]:
        x["director"] = ""
    return (
        " ".join(x["keywords"])
        + " "
        + " ".join(x["cast"])
        + " "
        + x["director"]
        + " "
        + " ".join(x["genres"])
    )
   


df2["soup"] = df2.apply(create_soup, axis=1)

- 아래 과정은 위와 동일

In [ ]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df2['soup'])

In [ ]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
# Reset index of our main DataFrame and construct reverse mapping as before
df2 = df2.reset_index()
indices = pd.Series(df2.index, index=df2['title'])

In [ ]:
get_recommendations('The Dark Knight Rises', cosine_sim2)

In [ ]:
get_recommendations('The Godfather', cosine_sim2)

## Collaborative Filtering

- Content Based Filtering는 특정 영화와 유사한 영화를 추천하기 때문에 다른 장르의 영화를 추천할 수 없음
- 개인화된 추천도 제공하지 않고, 같은 영화라면 모두에게 같은 추천을 제공

### User Based Filtering

- 유사한 사용자가 좋아하는 영화를 추천
- Pearson Correlation Score 또는 Cosine Similarity를 사용하여 유사도 측정 가능
- 사용자의 선호가 시간에 따라 변화되어 미리 계산된 유사도를 사용하기 어려움

### Item Based Collaborative Filtering

- 타겟 사용자가 평가한 영화의 평가를 기반으로 유사한 영화를 추천
- Item Based Collaborative Filtering이 더 정적임
- Scalability 문제: m 명의 사용자와 n 개의 아이템이 있다면, O(m * n) 시간이 소요됨 
- Sparsity 문제: 두 사용자가 같은 영화를 평가한 경우가 드뭄

#### Single Value Decomposition

- Scalability와 Sparsity 문제를 해결하기 위해 Latent Factor Model 사용
- 이를 Optimization Problem으로 지환하고, RMSE를 최소화하는 방향으로 학습

In [ ]:
import kagglehub

# Download latest version
path2 = kagglehub.dataset_download("rounakbanik/the-movies-dataset")

print("Path to dataset files:", path2)

In [ ]:
from surprise import Reader

reader = Reader()

ratings = pd.read_csv(f"{path2}/ratings_small.csv")
ratings.head()

In [ ]:
from surprise import Dataset, SVD, accuracy
from surprise.model_selection import KFold


data = Dataset.load_from_df(ratings[["userId", "movieId", "rating"]], reader)

kf = KFold(n_splits=5)

svd = SVD()

# 각 폴드에서 학습 및 평가
for trainset, testset in kf.split(data):
    svd.fit(trainset)  # 모델 학습
    predictions = svd.test(testset)  # 테스트 데이터 예측

    # RMSE와 MAE 출력
    rmse = accuracy.rmse(predictions, verbose=True)
    mae = accuracy.mae(predictions, verbose=True)

In [ ]:
ratings[ratings['userId'] == 1]

In [ ]:
svd.predict(1, 302, 3)